In [2]:
import numpy as np
import netCDF4 as ncdf
import matplotlib.pyplot as plt
import xarray as xr
import glob as glob
import os as os
import sys as sys
import pygrib
# from gridrad import *
#from cmpref import cmpref_mod as cmpref
from functools import partial
import time
import multiprocessing
from contextlib import contextmanager
from zipfile import *
import io
import dask.array as da
from datetime import datetime, time, timedelta
from time import sleep

_gravity = 9.81
_Rgas = 8.31446

def regrid_hrrr(f,date,hours,fins,outdir,suffix,prefix="HRRR",sw_corner=None, ne_corner=None, grib2_vars=None):    
    
    if not grib2_vars:
        grib2_vars = {     #  keys                 / No. of Dims /  Type   / bottomLevel / paramCategory / paramNumber
               'TEMP':     [{'shortName':'t','typeOfLevel':'hybrid'}],
               'P':        [{'shortName':'pres','typeOfLevel':'hybrid'}],
               'QVAPOR':   [{'shortName':'q','typeOfLevel':'hybrid'}],
               'W':        [{'shortName':'w','typeOfLevel':'hybrid'}],
               'U':        [{'shortName':'u','typeOfLevel':'hybrid'}],
               'V':        [{'shortName':'v','typeOfLevel':'hybrid'}],
               'GPH':      [{'shortName':'gh','typeOfLevel':'hybrid'}],
               'uphl_max_25': [{'name':'Updraft Helicity','stepType': 'max','topLevel':5000}],
               'uphl_max_02': [{'name':'Updraft Helicity','stepType': 'max','topLevel':2000,'bottomLevel':0}],
               'uphl_max_03': [{'name':'Updraft Helicity','stepType': 'max','topLevel':3000,'bottomLevel':0}],
              # 'UH':       [{'name':'Updraft Helicity','stepType': 'max'}],
               'refc':     [{'shortName':'refc'}],
               'refd':     [{'shortName':'refd','typeOfLevel': 'isothermal','stepType': 'instant'}],
               'orog':     [{'shortName':'orog','typeOfLevel':'surface'}],
              # 'REFL_10CM':[{'shortName':'refd','typeOfLevel':'hybrid'}],
              # 'VIL':      [{'shortName':'veril'}],
               'maxref_1km':[{"typeOfLevel":"heightAboveGround",'stepType': 'max','level':1000}],
               'tp':       [{'shortName':'tp','forecastTime':int(hours[f])-1}],
               'land':     [{'shortName':'lsm'}],
               'cape':     [{'shortName':'cape','typeOfLevel':'surface'}],
               'cin':      [{'shortName':'cin','typeOfLevel':'surface'}],
               'wmin':     [{'parameterCategory':2,'parameterNumber':221}],
               'srh_01' :  [{'shortName':'hlcy','topLevel':1000}],
               'srh_03' :  [{'shortName':'hlcy','topLevel':3000}],
               'et'  :     [{'parameterCategory':16,'parameterNumber':3}],
               'wmax':     [{'parameterCategory':2,'parameterNumber':220}],
               'T2'  :     [{'shortName':'2t'}],
               'Q2'  :     [{'parameterCategory':1,'parameterNumber':0,'level':2}],
               'TD2'  :    [{'shortName':'2d'}],
               'U10'  :    [{'shortName':'10u'}],
               'V10'  :    [{'shortName':'10v'}],
               'U80'  :    [{'shortName':'u','level':80}],
               'V80'  :    [{'shortName':'v','level':80}],
               'VUCSH':    [{'shortName':'vucsh'}],
               'VVCSH':    [{'shortName':'vvcsh'}]
               }
    #Extract the correct zip files
    file = fins[f]
    tmpdir = "%s/tmp"%outdir
    cmd = "mkdir -p %s"%tmpdir
    os.system(cmd)
    #with ZipFile(file) as z:
    #    zip_files = z.namelist()
    #    for file in zip_file:
    #        if not any (hour in file for h in hours): continue
    file_only = os.path.split(file)[1]    
    #print("reading input data into dataset")
    for k, key in enumerate(grib2_vars):
        #print(key)
        if k == 0:
            ds_in = xr.open_dataset(file,filter_by_keys=grib2_vars[key][0],engine='cfgrib',
                                    backend_kwargs={'indexpath': tmpdir + '/'+file_only+'.{short_hash}.idx'})
            
            fhour = float(ds_in.step)
            istring = '20'+file.split('/')[5]
            init_time = datetime.strptime(istring,'%Y%m%d%H')
            print(istring,init_time)
            #ts = init_time + np.timedelta64(fhour, 'h')
            #fcst_time = datetime.utcfromtimestamp(ts)
            fcst_time = init_time + timedelta(hours=int(fhour/3.6E12))
            fhour = fhour/3.6E12
            #print(ds_in.valid_time.time)
            #cst_time = ds_in[list(ds_in.keys())[0]].initial_time
            #fcst_time = datetime.strptime(fcst_time,'%m/%d/%Y (%H:%M)')
            keys = list(ds_in.keys())
            shape = np.shape(ds_in[keys[0]])
            pshape = [shape[2],shape[1],shape[0]]
            p = np.broadcast_to(ds_in.isobaricInhPa.values*100.0,pshape)
            p = np.transpose(p)
            attrs = ds_in[keys[0]].attrs
            attrs['GRIB_shortname'] = 'pres'
            attrs['GRIB_name'] = 'Pressure'
            attrs['long_name'] = 'Pressure'
            attrs['GRIB_unts'] = 'Pa'
            attrs['units'] = 'Pa'
            p = xr.DataArray(p,dims = ds_in[keys[0]].dims,
                             coords=ds_in[keys[0]].coords, attrs=attrs)
            ds_in['p'] = p
            #print(ds_in.valid_time.time)
            #cst_time = ds_in[list(ds_in.keys())[0]].initial_time
            #fcst_time = datetime.strptime(fcst_time,'%m/%d/%Y (%H:%M)')
        elif key=='P':
            continue
        elif key=='W':
            ds = xr.open_dataset(file,filter_by_keys=grib2_vars[key][0],engine='cfgrib',
                                 backend_kwargs={'indexpath': tmpdir + '/'+file_only+'.{short_hash}.idx'})
            wz = np.array(-ds.w.values / ( (_gravity * ds_in.pres.values) / (_Rgas * ds_in.t.values) ) )
            ds.w.values = wz
            attrs = ds_in.t.attrs
            attrs['GRIB_shortname'] = 'wz'
            attrs['GRIB_name'] = 'Geometric Vertical Velocity'
            attrs['long_name'] = 'Geometric Vertical Velocity'
            attrs['GRIB_unts'] = 'm s**-1'
            attrs['units'] = 'm s**-1'
            ds = ds.rename({'w':'wz'})
            ds.wz.attrs = attrs
            ds_in = xr.merge([ds_in,ds],join='override')
        elif key=='REFL_10CM':
            ds = xr.open_dataset(file, filter_by_keys={'typeOfLevel':'hybrid'}, engine='cfgrib',
                                        backend_kwargs={'indexpath': tmpdir + '/'+file_only+'.{short_hash}.idx'})
            nz,ny,nx = np.shape(ds.q.values)
            # grb_var = cmpref.calcrefl10cm(ds.q.values, ds.clwmr.values, ds.rwmr.values, ds.snmr.values, ds.grle.values, 
            #                               ds.t.values, ds.pres.values, nz, nx, ny)
            attrs = ds_in.refc.attrs
            attrs['GRIB_typeOfLevel'] = 'hybrid'
            attrs['GRIB_name'] = 'Simulated 10cm Radar Reflectivity'
            attrs['GRIB_shortname'] = 'refl10cm'
            attrs['long_name'] = 'Simulated 10cm Radar Reflectivity'
            refl = xr.DataArray(grb_var, dims = ds_in.t.dims, 
                                coords=ds_in.t.coords,
                                attrs=attrs)
            ds_in['refl10cm'] = refl
        elif 'uphl_max' in key:
            ds = xr.open_dataset(file,filter_by_keys=grib2_vars[key][0],engine='cfgrib',
                                        backend_kwargs={'indexpath': tmpdir + '/'+file_only+'.{short_hash}.idx'})
            ds = ds.rename({'uphl':key})
            ds_in = xr.merge([ds_in,ds],join='override',compat='override')
        elif 'srh' in key:
            ds = xr.open_dataset(file,filter_by_keys=grib2_vars[key][0],engine='cfgrib',
                                        backend_kwargs={'indexpath': tmpdir + '/'+file_only+'.{short_hash}.idx'})
            ds = ds.rename({'hlcy':key})
            ds_in = xr.merge([ds_in,ds],join='override',compat='override')
        elif key == 'UH':
            continue
        else:
            ds = xr.open_dataset(file,filter_by_keys=grib2_vars[key][0],engine='cfgrib',
                                        backend_kwargs={'indexpath': tmpdir + '/'+file_only+'.{short_hash}.idx'})
            if key=='U80':
                ds = ds.rename({'u':'u80'})
            if key=='V80':
                ds = ds.rename({'v':'v80'})
            if key=='Q2':
                ds = ds.rename({'q':'q2'})
            if list(ds.variables)[-1] == 'unknown':
                ds = ds.rename({'unknown':key})
            if list(ds.variables)[-1] == 'veril':
                ds = ds.rename({'veril':'veril_orig'})
            if 'heightAboveGround' in ds.coords and 'heightAboveGround' in ds_in.coords:
                ds_in = ds_in.rename({'heightAboveGround':'heightAboveGround1'})
                ds = ds.rename({'heightAboveGround':'heightAboveGround2'})
                nhagl = 2
            if 'heightAboveGround' in ds.coords and 'heightAboveGround1' in ds_in.coords:
                nhagl += 1
                ds = ds.rename({'heightAboveGround':'heightAboveGround%i'%nhagl})

            ds_in = xr.merge([ds_in,ds],join='outer',compat='override')
            
    if not (sw_corner is None) and not (ne_corner is None):
        #print("Crop lat/lons")
        ds_in = ds_in.rename({'x':'west_east','y':'south_north'})
        lats = ds_in.latitude.values
        lons = ds_in.longitude.values
        if np.max(lons) > 180.0: lons = lons - 360.0
        crop = xr.DataArray(np.logical_and(np.logical_and(lats>sw_corner[0],
                             lats<ne_corner[0]),np.logical_and(lons>sw_corner[1],lons<ne_corner[1])),
                            dims=['south_north','west_east'])
        ds_out = ds_in.where(crop,drop=True)
    else:
        ds_out = ds_in
    
    if 'REFL_10CM' in list(grib2_vars):
        #print("Compute auxillary variables from REFL_10CM")
        
        #echo top
        nz,ny,nx = np.shape(ds_out.t.data)
        echo_hgt_domain = np.empty([3,ny,nx])
        for d, db in enumerate([30.0, 50.0, 60.0]):
            refl_db = np.ma.masked_where(ds_out.refl10cm < db, ds_out.refl10cm)
            mask_hgt = np.ma.masked_where(np.ma.getmask(refl_db), ds_out.gh)

            mask_hgt = mask_hgt.filled(0)
            echo_hgt_domain[d,:] = np.nanmax(mask_hgt, axis=0)
            attrs = ds_out.t.attrs
            attrs['GRIB_shortname'] = 'retop'
            attrs['GRIB_name'] = 'Simulated Radar Echo Top'
            attrs['long_name'] = 'Simulated Radar Echo Top'
            attrs['GRIB_unts'] = 'm'
            attrs['units'] = 'm'
            attrs['GRIB_typeOfLevel'] = 'heightAboveGroundLevel'
            attrs['GRIB_cfVarName'] = 'et'
            attrs['GRIB_cfName'] = 'echo_top'
            
        echo_hgt = xr.DataArray(echo_hgt_domain, dims = ["heightAboveGroundLevelET","south_north","west_east"],
                                coords=dict(latitude = (["south_north","west_east"], ds_out.latitude.values,
                                                        {'units':'degrees_north'}),
                                            longitude = (["south_north","west_east"], ds_out.longitude.values, 
                                                         {'units':'degrees_east'}),
                                            heightAboveGroundLevelET = (["heightAboveGroundLevelET"], [30.0, 50.0, 60.0], 
                                                                       {'units':'dBZ'})), 
                                attrs=attrs)
        ds_out['et'] = echo_hgt
        
        # refl at -10C
        ind_10c = np.argmin(np.abs(ds_out.t.values-263.15),axis=0)
        idx = np.indices(ind_10c.shape)
        ref_10c = ds_out.refl10cm.values[ind_10c,idx[0],idx[1]]
        #ref_10c = np.take_along_axis(ds_out.refl10cm.values,ind_10c,axis=0)
        
        
        attrs['GRIB_shortname'] = 'refd'
        attrs['GRIB_name'] = 'Isothermal Reflectivity at -10C'
        attrs['long_name'] = 'Isothermal Reflectivity at -10C'
        attrs['GRIB_unts'] = 'dBZ'
        attrs['units'] = 'dBZ'
        attrs['GRIB_typeOfLevel'] = 'heightAboveGroundLevel'
        attrs['GRIB_cfVarName'] = 'refd'
        attrs['GRIB_cfName'] = 'reflectivity_10c'
        ref_10c = xr.DataArray(ref_10c, dims = ds_out.orog.dims, coords = ds_out.orog.coords, attrs = attrs)
        ds_out['refd'] = ref_10c
        
        # height of composite reflectivity
        ind_refc = np.argmax(ds_out.refl10cm.values,axis=0)
        idx = np.indices(ind_refc.shape)
        hgt_refc = ds_out.gh.values[ind_refc,idx[0],idx[1]]
        #hgt_refc = np.take_along_axis(ds_out.gh.values,ind_refc,axis=0)
        attrs['GRIB_shortname'] = 'zrefc'
        attrs['GRIB_name'] = 'Height of Composite Reflectivity Above Sea Level'
        attrs['long_name'] = 'Height of Composite Reflectivity Above Sea Level'
        attrs['GRIB_unts'] = 'm'
        attrs['units'] = 'm'
        attrs['GRIB_typeOfLevel'] = 'heightAboveGroundLevel'
        attrs['GRIB_cfVarName'] = 'zrefc'
        attrs['GRIB_cfName'] = 'height_refc'
        hgt_refc = xr.DataArray(hgt_refc, coords = ds_out.orog.coords, dims = ds_out.orog.dims, attrs = attrs)
        ds_out['zrefc'] = hgt_refc
        
        # vil
        dgph = ds_out.gh.values[1:,:] - ds_out.gh.values[:-1,:]
        vil = 0.00344 * (10.0**(0.05 * (ds_out.refl10cm.values[1:,:] + ds_out.refl10cm.values[:-1,:]))) **0.57143 * dgph /1000.0
        vil = np.insert(vil,0,0.00344 * ds_out.refl10cm.values[0,:] **0.57143 * ds_out.gh.values[0,:] /1000.0,axis=0)
        vil = np.nansum(vil,axis=0)
        attrs = ds_out.orog.attrs
        attrs['GRIB_name'] = 'Simulated Radar Vertically Integrated Liquid'
        attrs['GRIB_shortname'] = 'vil'
        attrs['long_name'] = 'Simulated Radar Vertically Integrated Liquid'
        attrs['GRIB_unts'] = 'kg m^-3'
        attrs['units'] = 'kg m^-3'
        attrs['GRIB_cfVarName'] = 'veril'
        attrs['GRIB_cfName'] = 'vert_int_liquid'
        vil = xr.DataArray(vil, dims = ds_out.orog.dims, 
                            coords=ds_out.orog.coords,
                            attrs=attrs)
        ds_out['veril'] = vil 
        
    # if 'UH' in list(grib2_vars):
    #     print("Compute instantaneous UH")
    #     temp = np.empty_like(np.transpose(ds_out.orog.values))
    #     temp = calc_uh(np.transpose(ds_out.gh.values),3000.0,3000.0, 2000.0, 5000.0, np.transpose(ds_out.u.values), 
    #             np.transpose(ds_out.v.values),np.transpose(ds_out.wz.values), temp)
    #     print(np.max(temp))
        # ds_out['uphl'] = xr.DataArray(np.transpose(temp),coords=ds_out.orog.coords,dims=ds_out.orog.dims, 
        #                               attrs = ds_out.uphl_max.attrs)

    print("%s Creating dataset and writing to file"%date )
    outfname = "%s/%s_%s_%s.nc"%(outdir,fcst_time.strftime('%Y%m%d%H%M'),prefix,suffix)
    ds_out.attrs['date']       = date
    ds_out.attrs['time']       = fcst_time.strftime('%H:%M')
    ds_out.attrs['fhour']      = fhour
    ds_out.attrs['gridType']   = 'HRRR_%s'%suffix
    ds_out.attrs['DateTime']   = datetime.now().strftime("%Y%m%d_%H:%M:%S")
    ds_out.attrs['TimeStamp']  = datetime.timestamp(datetime.now())

    ds_out.to_netcdf(outfname, mode='w') 

In [3]:
from herbie import Herbie
H = Herbie(
    "2023-02-15 06:00",
    model="hrrr",
    product="sfc",
    fxx=0,
        )
q = "TMP:2 m above"
ds = H.xarray(q)
ds

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Feb-15 06:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


<xarray.Dataset>
Dimensions:              (y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 2023-02-15T06:00:00
    step                 timedelta64[ns] 00:00:00
    heightAboveGround    float64 2.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           datetime64[ns] 2023-02-15T06:00:00
Dimensions without coordinates: y, x
Data variables:
    t2m                  (y, x) float32 291.0 291.0 291.0 ... 268.9 269.0 269.0
    gribfile_projection  object None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 sfc
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr....
    local_grib:              /Users/johnlawson/data/hrrr/20230215/subset_f7ef...
    searchString:            TMP:2 m above

In [3]:
# def regrid_hrrr(f,date,hours,fins,outdir,suffix,prefix="HRRR",sw_corner=None, ne_corner=None, grib2_vars=None):    
regrid_hrrr(dsm)